In [59]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [61]:
cap = cv2.VideoCapture(0)

In [55]:
eye = cv2.CascadeClassifier('DATA/HaarCascades/haarcascade_eye_tree_eyeglasses.xml')

In [43]:
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

eyes = eye.detectMultiScale(gray, 1.1, 3)

if len(eyes)>0:
    x,y,w,h = eyes[0]
    eye_params = (max(x-10, 0), max(y-10, 0), w+20, h+20) 

    
    roi = frame[y:y+h, x:x+w]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
    cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

    term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        ret, track = cv2.meanShift(dst, eye_params, term_criteria)

        x, y, w, h = track

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.imshow("Tracking", frame)

        if cv2.waitKey(1) & 0xFF == 27: 
            break

cap.release()
cv2.destroyAllWindows()

In [57]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

eyes = eye_cascade.detectMultiScale(gray, 1.1, 3)

eye_params_list = []
roi_hist_list = []

if len(eyes) > 0:
    for (x, y, w, h) in eyes:
        x, y, w, h = max(x-10, 0), max(y-10, 0), w+20, h+20
        eye_params_list.append([x, y, w, h])  # Store each eye separately

        roi = frame[y:y+h, x:x+w]
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

        roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
        cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
        roi_hist_list.append(roi_hist)  # Store histograms for each eye

term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    for i in range(len(eye_params_list)):  # Track each eye separately
        dst = cv2.calcBackProject([hsv], [0], roi_hist_list[i], [0, 180], 1)
        ret, track = cv2.meanShift(dst, tuple(eye_params_list[i]), term_criteria)

        # Update bounding box position
        eye_params_list[i][0], eye_params_list[i][1] = track[0], track[1]

        x, y, w, h = eye_params_list[i]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("Eye Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


In [62]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

eye_params_list = []
roi_hist_list = []

if len(faces) > 0:
    for (fx, fy, fw, fh) in faces:
        roi_gray = gray[fy:fy+fh, fx:fx+fw]  
        roi_color = frame[fy:fy+fh, fx:fx+fw]

        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=8, minSize=(20, 20))

        for (ex, ey, ew, eh) in eyes:
            x, y, w, h = fx + ex, fy + ey, ew, eh

            x, y, w, h = max(x-10, 0), max(y-10, 0), w+20, h+20
            eye_params_list.append([x, y, w, h])  

            roi = frame[y:y+h, x:x+w]
            hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

            roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
            cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
            roi_hist_list.append(roi_hist)  # Store histograms for each eye

term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    for i in range(len(eye_params_list)):  
        dst = cv2.calcBackProject([hsv], [0], roi_hist_list[i], [0, 180], 1)
        ret, track = cv2.meanShift(dst, tuple(eye_params_list[i]), term_criteria)

        eye_params_list[i][0], eye_params_list[i][1] = track[0], track[1]

        x, y, w, h = eye_params_list[i]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("Eye Tracking", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()
